In [4]:
# -*- coding: utf-8 -*-

import keras
from keras import models
import cv2
import numpy as np
from moviepy.editor import *
import os
import glob
import json
import collections as cl
from copy import deepcopy
import moviepy.editor as mpe


class SynthesizeNetwork:

    def __init__(self):
        self.video_path = "test_video.mp4"
        self.category_path = "model_data/0317_7model_1.h5"
        self.num_cut = 1 #フレーム間の数　１なら全フレーム取得する

    def load_model(self, category_path=None):
        #モデルのロードをするだけの関数
        if not category_path:
            category_path = self.category_path
        self.category_model = models.load_model(category_path)


    def movie_to_image(self, video_path=None, num_cut=None, size=(150, 150)):
        #動画のパスを渡すとnum_cut(フレーム)ごとの画像をarrayにして返す

        if not num_cut:
            num_cut = self.num_cut
        if not video_path:
            video_path = self.video_path

        video = []
        #size = (150, 150)
        frame_count = 0
        # キャプチャ動画読み込み（キャプチャ構造体生成）
        capture = cv2.VideoCapture(video_path)

        # フレーム画像がある限りループ
        while(capture.isOpened()):
            # フレーム画像一枚取得
            ret, frame = capture.read()
            if ret == False:
                break

            # 指定した数だけフレーム画像を間引いて保存
            if frame_count % num_cut == 0:
                frame = cv2.resize(frame, size)
                #print(frame.shape)
                video.append(frame)

            frame_count += 1

        # キャプチャ構造体開放
        capture.release()
        video_ar = np.array(video)

        #配列が空ならエラーにする
        if not np.any(video_ar):
            raise Exception('配列が空です!')

        return video_ar


    def background_subtraction(self, video_ar):
        #背景差分をとる関数
        sub = video_ar[:-1] - video_ar[1:]
        #maeの計算 shape(sample数-1,)
        subtraction_ar = np.mean(np.abs(sub.reshape(len(sub), -1)), axis=1)
        return subtraction_ar


    def get_thresh_subtraction_index(self, subtraction_ar):
        """
        背景差分を渡すと、シーン分割のインデックスを返す関数

        scene_ar:array shape(num_scene, )
            0から始まる。各値はシーンの開始フレーム, 動画の最終フレーム
        """

        THRESH = 0.7 #差分の閾値

        scene_ar = np.where(subtraction_ar >= THRESH)[0]
        scene_ar += 1  #差分なのでシーンの分かれ目はプラス１になる
        scene_ar = np.insert(scene_ar, 0, 0)  #ぜろを先頭に挿入

        #秒数が近いインデックスは消す 閾値15フレーム
        for i in range(len(scene_ar)):
            try:
                while True:
                    if (scene_ar[i + 1] - scene_ar[i]) <= 15:
                        scene_ar = np.delete(scene_ar, i + 1)
                    else:
                        break
            except IndexError:
                break

        #動画の最後からフレームが空いてないとそのインデックスは消す
        while True:
            if (len(subtraction_ar) - scene_ar[-1]) <= 15:
                scene_ar = np.delete(scene_ar, -1)
            else:
                break

        scene_ar = np.append(scene_ar,len(subtraction_ar))  #動画の最終フレーム数を最後に挿入
        return scene_ar


    def get_choice_subtraction_index(self, subtraction_ar, num_scene):
        """
        背景差分を渡すと、シーン分割のインデックスを返す関数

        num_scene: int
            シーンの分かれ目の欲しい数　2ならシーンとしては3になります
        scene_ar:array shape(num_scene, )
            0から始まる。各値はシーンの開始フレーム, 動画の最終フレーム
        """
        thresh_frame = 90 #間を開けるフレーム数の閾値

        #ゼロなら処理終了
        if num_scene == 0:
            scene_ar = np.array([0, len(subtraction_ar)])
            return scene_ar

        arg_ind = np.argsort(-subtraction_ar)
        kari_num_scene = 10
        #kari_num_sceneが希望する分割数より少ない場合増やす
        if num_scene > kari_num_scene:
            kari_num_scene = num_scene + 5

        while len(arg_ind) >= kari_num_scene:
            #とりあえず差分上位１０個だけ抜き出す,足りないならループで増やす
            arg_kari_ind = arg_ind[:kari_num_scene]
            sort_ind = np.sort(arg_kari_ind)

            #秒数が近いインデックスは消す 閾値15フレーム
            for i in range(len(sort_ind)):
                try:
                    while True:
                        if (sort_ind[i + 1] - sort_ind[i]) <= thresh_frame:
                            sort_ind = np.delete(sort_ind, i + 1)
                        else:
                            break
                except IndexError:
                    break

            #動画の最初から0.5秒空いてないとそのインデックスは消す
            try:
                while True:
                    if sort_ind[0] <= thresh_frame:
                        sort_ind = np.delete(sort_ind, 0)
                    else:
                        break
            
            #動画の最後から0.5秒空いてないとそのインデックスは消す
                while True:
                    if (len(subtraction_ar) - 1 - sort_ind[-1]) <= thresh_frame:
                        sort_ind = np.delete(sort_ind, -1)
                    else:
                        break
            except IndexError:
                    break

            #num_sceneより多くシーンが抜き出せていればbreakする
            if len(sort_ind) >= num_scene:
                break
            else:
                kari_num_scene += 5

        #昇順になっているsort_indを差分降順に直す
        scene_ar = np.zeros(len(arg_kari_ind)) - 99
        for i in sort_ind:
            id = np.where(arg_kari_ind == i)[0]
            scene_ar[id] = i
        scene_ar = np.delete(scene_ar, np.where(scene_ar == -99))
        scene_ar += 1  #差分なのでシーンの分かれ目はプラス１になる
        scene_ar = np.sort(scene_ar[:num_scene])  #シーンを取り出し後昇順に並び替える
        scene_ar = np.insert(scene_ar, 0, 0)  #ぜろを先頭に挿入
        scene_ar = np.append(scene_ar,len(subtraction_ar))  #動画の最終フレーム数を最後に挿入

        return scene_ar

    def classify_movie_to_json(self, image_ar, scene_ar, video_path=None, json_name=None):
        """
        素材一覧のjsonファイルを作る
        """
        if not json_name:
            json_name = "material" #jsonファイルの名前

        if not video_path:
            video_path = self.video_path  #編集したい動画のパス
        print("try前")
        try:
            f = open(json_name + ".json", 'r')
            json_data = json.load(f)
            jsn = cl.OrderedDict(json_data)
            num_count = max(list(map(int, jsn.keys()))) + 1

        except:
            #collections.OrderedDictで順序を指定した辞書を作成
            jsn = cl.OrderedDict()
            num_count = 0
        print("try後")

        image_ar = image_ar / 255.
        print(image_ar.shape)
        pred_proba = self.category_model.predict(image_ar, verbose=1)
        print("pred完了")

        for i in range(len(scene_ar)-1):
            arange_ar = np.arange(scene_ar[i], scene_ar[i + 1]).astype("int32")
            clas = np.argmax(np.sum(pred_proba[arange_ar, :], axis=0))
            pred_proba_one_class = pred_proba[arange_ar, clas].reshape(-1).tolist()
            #辞書の作成
            data = cl.OrderedDict()
            data["path"] = video_path
            data["start"] = int(scene_ar[i])
            data["stop"] = int(scene_ar[i + 1])
            data["class"] = int(clas)
            data["classs_proba"] = pred_proba_one_class
            jsn[num_count] = data
            num_count += 1
            print(str(i) + "シーン目　素材分類完了")

        #json.dump関数でファイルに書き込む 二回やらないとなぜかうまく書き込めない
        fw = open(json_name + ".json", 'w')
        json.dump(jsn, fw, indent=4)
        fw = open(json_name + ".json", 'w')
        json.dump(jsn, fw, indent=4)
        print("json書き出し完了：")


    def init_material_json(self):
        json_name = "material"
        fw = open(json_name + ".json", 'w')
        json.dump("", fw, indent=4)
        fw = open(json_name + ".json", 'w')
        json.dump("", fw, indent=4)
        print("json初期化完了：")

    def get_num_scene(self, video_path):
        """
        動画から取り出すシーン数を決める
        """
        image_ar = self.movie_to_image(video_path)
        video_span = image_ar.shape[0]/30

        if (video_span//9) -1 < 1:
            num_scene = 0
        elif 1 <= (video_span//9) -1 < 19:
            num_scene = (video_span//9) -1

        else:
            num_scene = 19
        return num_scene


    def get_scene_movie_json(self,video_path):
        """
        動画をシーンで分類しジェイソンファイルを作成するまでまとめたやつ
        """
        image_ar = self.movie_to_image(video_path)
        num_scene = self.get_num_scene(video_path)
        print("ビデオをイメージ化済み")
        sabun_ar = self.background_subtraction(image_ar)
        scene_ar = self.get_choice_subtraction_index(sabun_ar, int(num_scene))
        print("分類開始")
        #self.scene_to_movie(scene_ar, video_path)
        self.classify_movie_to_json(image_ar, scene_ar, video_path)
        


    def synthesizer(self, material_json, map_file=None):
        """
        動画合成
        mapfileをどうするか
        """

        #今後map_classとmap_secondはファイルを用意して読み込む形にする
        map_class = [1, 1, 1, 0, 1, 2, 1, 1, 1, 5, 0, 5, 0, 0, 0, 5, 1, 1, 1, 1]
        map_second = [0.,0.86666667,2.8,6.56666667,9.76666667,12.16666667,
                    12.83333333, 16.66666667, 17.2,        22.1,        24.73333333, 26.96666667,
                    27.6,        29.8,        41.23333333, 43.16666667, 46.66666667, 51.23333333,
                    52.86666667, 57.76666667, 59.96666667]

        num_map = len(map_class)
        fw = open(material_json, 'r')
        material = json.load(fw) #辞書
        material_class = np.array([material[k]["class"] for k in material.keys()]) #数字のarray
        material_length_frame = np.array(
            [len(material[k]["classs_proba"]) for k in material.keys()])  #動画ごとのフレーム数

        Exclusion_ind_list = []
        export_ar = np.empty((0,3))

        for i in range(num_map):
            kaburimode = False
            material_del = deepcopy(material)
            print(material_del.keys())
            for d in Exclusion_ind_list:
                print(d)
                try:
                    del material_del[str(int(d))]
                except KeyError:
                    continue
            material_class_del = np.array([
                material_del[k]["class"] for k in material_del.keys()])  #数字のarray
            material_length_frame_del = np.array([
                len(material_del[k]["classs_proba"]) for k in material_del.keys()]) #動画ごとのフレーム数

            map_num_frame = int((map_second[i + 1] - map_second[i])*30)
            c_ind = np.where(material_class_del == map_class[i])[0]  #クラスがあるか
            l_ind = np.where(material_length_frame_del > map_num_frame)[0]  #長さが足りているか

            if not np.any(l_ind):  #ないなら被りおk状態になる
                print("被りありモード")
                kaburimode = True
                c_ind = np.where(material_class == map_class[i])[0]
                l_ind = np.where(material_length_frame > map_num_frame)[0]
            print(map_num_frame)
            ##ここから合致した動画を探す
            if not np.any(l_ind):
                print("素材の長さが足りないです　処理終了")
                break

            if not np.any(c_ind):
                if int(map_class[i]) in [2, 3, 4, 5, 6]:  #if ブツ撮り
                    print("ブツ撮りのその他のクラスへ割り当て")
                    if kaburimode:
                        c_ind = np.where(material_class >= 2)[0]
                    else:
                        c_ind = np.where(material_class_del >= 2)[0]

            if not np.any(c_ind):
                print("クラスなし適当に割り当て")
                c_ind = l_ind

            if np.any(c_ind) and np.any(l_ind):
                find_list = []
                for c in c_ind:
                    if c in l_ind:
                        find_list += [c]
                find_ar = np.array(find_list)

            if not np.any(find_ar):
                print("find_arなし適当に割り当て")
                find_ar = l_ind

            if kaburimode:
                sort_ind = np.argsort(material_length_frame)
            else:
                sort_ind = np.argsort(material_length_frame_del)

            optimum_ar = np.empty((0,2))
            for j in find_ar:
                for l, k in enumerate(sort_ind):
                    if j == k:
                        optimum_ar =np.vstack((optimum_ar, np.array([l, k])))
                        break
            optimum_ind = optimum_ar[np.argmin(optimum_ar[:, 0]), 1]

            if kaburimode:
                optimum_ind = str(int(optimum_ind))
            else:
                optimum_ind = list(material_del.keys())[int(optimum_ind)]

            ##ここから最適なフレームを抜き出す
            class_proba_ar = np.array(material[optimum_ind]["classs_proba"])
            print(len(class_proba_ar))
            print("map_num_frame")
            print(map_num_frame)
            ind_ar = np.array([np.arange(i, i + map_num_frame) for i in np.arange(0, len(class_proba_ar) - map_num_frame)])  #最後オーバーすると怖いので一応マイナス１
            start = (np.argmax(np.sum(class_proba_ar[ind_ar], axis=1))
                    + material[optimum_ind]["start"]) / 30  #一番確率が高いシーケンスの始まり秒
            end = start + map_second[i + 1] - map_second[i]
            #arrayに保存しておく
            export_ar = np.vstack((export_ar, np.array([optimum_ind, start, end]).reshape(1, 3)))
            #使用ずみの素材は除くためのリスト
            Exclusion_ind_list += [optimum_ind]

        #書き出し
        cliplist = []
        for i in range(len(export_ar)):
            print(i)
            cliplist += [
                VideoFileClip(
                    material[str(export_ar[i, 0])]["path"],
                    audio=False).subclip(
                        float(export_ar[i, 1]), float(export_ar[i, 2]))
            ]
        final_clip = concatenate_videoclips(cliplist)  #ビデオの接続
        audioclip = AudioFileClip("kzk_wedding1.mp3")
        final_clip = final_clip.set_audio(audioclip)
        final_clip.write_videofile("a_test_mv.mp4", fps=29)  # Many options...#書き込み


Using TensorFlow backend.


Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)1548288/45929032 bytes (3.4%)5136384/45929032 bytes (11.2%)8871936/45929032 bytes (19.3%)12607488/45929032 bytes (27.4%)16351232/45929032 bytes (35.6%)20078592/45929032 bytes (43.7%)23912448/45929032 bytes (52.1%)27729920/45929032 bytes (60.4%)31342592/45929032 bytes (68.2%)35053568/45929032 bytes (76.3%)38830080/45929032 bytes (84.5%)42557440/45929032 bytes (92.7%)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# !pwd
# !ls
cd /content/gdrive/My Drive/wedding_test

/content/gdrive/My Drive/wedding_test


In [0]:
# video_paths = ["C0056.MP4","C0058.MP4","C0059.MP4","C0060.MP4","C0061.MP4","C0063.MP4","C0064.MP4","C0065.MP4","C0066.MP4","C0067.MP4","C0068.MP4"]

# # 書き出し
# cliplist = []

# cliplist += [VideoFileClip(path,audio=False) for path in video_paths]

# final_clip_s = concatenate_videoclips(cliplist)  #ビデオの接続
# # final_clip.write_videofile("combine_movie.mp4", fps=29)  # Many options...#書き込み

In [0]:
# import moviepy.video.fx.all

# final_clip = moviepy.video.fx.all.resize(final_clip_s, height=150, width=150)
# final_clip = final_clip.iter_frames()

In [0]:
# import numpy as np

# final_clip_list = np.zeros([1800, 150, 266, 3])
# for i, clip in enumerate(final_clip):
#   if i < 1800:
#     final_clip_list[i] = clip
    
#   else:
#     break
  
# #   final_clip_list.append(list(i))

In [0]:
# # for i in range(5):
# #   final_clip_list.append(i)

# final_clip_list

[]

In [0]:
# import SynthesizeNetwork
wed = SynthesizeNetwork()

# Json

In [6]:
wed.init_material_json()

json初期化完了：


In [7]:
wed.load_model("0317_7model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
# video_path = "combine_movie.mp4"

# wed.get_scene_movie_json(final_clip)

In [8]:
video_paths = ["M&Y,Opening,vol1.MP4"]

for path in video_paths:
  wed.get_scene_movie_json(path)

ビデオをイメージ化済み
分類開始
try前
try後
(4590, 150, 150, 3)
4590/4590 [==============================] - 29s 6ms/step
pred完了
0シーン目　素材分類完了
1シーン目　素材分類完了
2シーン目　素材分類完了
3シーン目　素材分類完了
4シーン目　素材分類完了
5シーン目　素材分類完了
6シーン目　素材分類完了
7シーン目　素材分類完了
8シーン目　素材分類完了
9シーン目　素材分類完了
10シーン目　素材分類完了
11シーン目　素材分類完了
12シーン目　素材分類完了
13シーン目　素材分類完了
14シーン目　素材分類完了
15シーン目　素材分類完了
16シーン目　素材分類完了
json書き出し完了：


In [0]:
import time
# video_paths = ["kzk_wedding8.mp4"]
# video_paths = ["kzk_wedding1.mp4","kzk_wedding2.mp4","kzk_wedding3.mp4","kzk_wedding4.mp4","kzk_wedding5.mp4","kzk_wedding6.mp4","kzk_wedding7.mp4", "kzk_wedding8.mp4", "kzk_wedding9.mp4"]

for i in range(52):
  try:
    wed.get_scene_movie_json("cut{}.mp4".format(i))
  except:
    continue

In [0]:
jsonpath = "material.json"

In [11]:
wed.synthesizer(jsonpath)

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
26
111
map_num_frame
26
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
14
57
119
map_num_frame
57
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
14
6
113
140
map_num_frame
113
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
14
6
3
95
108
map_num_frame
95
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
14
6
3
7
72
300
map_num_frame
72
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
14
6
3
7
13
19
ブツ撮りのその他のクラスへ割り当て
232
map_num_frame
19
dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'])
14
6
3
7
13
0
115
319
map_num_frame
115
dict_keys(['0', '1', '2', '3', '

100%|██████████| 1326/1326 [00:01<00:00, 666.17it/s]

[MoviePy] Done.
[MoviePy] Writing video a_test_mv.mp4



100%|██████████| 1740/1740 [04:05<00:00,  4.96it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: a_test_mv.mp4 

